Install Pytorch and fastbook

In [2]:
!pip install torch
!pip install -Uqq fastbook
import fastbook

     |████████████████████████████████| 727kB 27.3MB/s 
     |████████████████████████████████| 1.2MB 38.9MB/s 
     |████████████████████████████████| 194kB 57.2MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 61kB 10.5MB/s 
     |████████████████████████████████| 61kB 10.5MB/s 


Import fastbook, colab, image display library

In [3]:
from fastbook import *
from google.colab import files
from IPython.display import Image
from IPython.display import HTML

Get images from github

In [ ]:
!npx degit github:GemmyTheGeek/FoodyDudy FoodyDudy --force
path = Path('FoodyDudy/images')

Create
 Datablock

In [ ]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #x - image; y - single class
    get_items=get_image_files, #get image
    splitter=GrandparentSplitter(), #use parent folder as train-valid split
    get_y=parent_label, #use parent folder as label
    #two choices for resizing and rationale
    #squishing to prevent cropping places without chips/raisins
    # item_tfms=Resize(224, method=ResizeMethod.Squish), 
    # #cropping to preserve image quality; tried and doesn't work - peaked at 0.85 val acc
    # item_tfms=RandomResizedCrop(512),
    batch_tfms=aug_transforms(size=512)
    )
dls = dblock.dataloaders(path, bs=64) #batch size = 64

In [ ]:
dls.train.show_batch(max_n=70,nrows=10)

Training

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(10)

Creating a Confusion Matrix

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10,10))

Displaying Food Chart

In [ ]:
Image('47thaifood.png')

Prints Top Loss

In [ ]:
interp.plot_top_losses(100, nrows=10)

Saving the model

In [ ]:
learn.save('model')

Exporting the model

In [ ]:
exportfilename='export.pkl'
learn.export(exportfilename)
path = Path()
path.ls(file_exts='.pkl')

Creating the datablock that is used for the test set

In [ ]:
dblock2 = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #x - image; y - single class
    get_items=get_image_files, #get image
    splitter=GrandparentSplitter(valid_name='test'), #use parent folder as train-test split
    get_y=parent_label, #use parent folder as label
    batch_tfms=aug_transforms(size=512)
    )
dls2 = dblock2.dataloaders(path, bs=64) #batch size = 64

learn = cnn_learner(dls2, resnet34, metrics=error_rate).to_fp16()
#load trained model
learn.load('model');

Creating a Confusion Matrix for test set

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10,10))

Prints out classification Report

In [ ]:
interp.print_classification_report() 

Download the model into local drive

In [ ]:
files.download(exportfilename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Upload Thai food images for testing

In [ ]:
uploaded = files.upload()

Displaying the test image

In [ ]:
imagetotest='test4.jpg'
Image(imagetotest)

The model tries to predict the image

In [ ]:
learn.predict(imagetotest)
result=learn.predict(imagetotest)
predictcat=int(result[0])
predictindex=int(result[1])
predictpct=result[2][predictindex]
predictcat
"Cat=" + str(predictcat) + " : %=" + str(float(predictpct)*100)

Upload previous exported model here

In [ ]:
uploaded = files.upload()
path.ls(file_exts='.pkl')

Learns from the exported model

In [ ]:
learn_inf = load_learner('resnet34-10.pkl')

Predicting test image

In [ ]:
learn_inf.predict('test4.jpg')

Presentation Video!

In [5]:
youtubeid='EEoyMhOuCAU'
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/' + youtubeid + '" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')